In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
#apply to our set of occupations
bls = pd.read_csv('../results/data_out/occ_names_bls_minor_major.csv', index_col=0)

In [ ]:
activities = pd.read_excel('../data/skill_Bowen/Work Activities.xlsx')
context = pd.read_excel('../data/skill_Bowen/Work Context.xlsx')
abilities = pd.read_excel('../data/skill_Bowen/Abilities.xlsx')

In [ ]:
cmmv =  pd.read_excel('../data/skill_Bowen/CMMV.xlsx')

Routine index (RTI index)	(Autor and Dorn (2013))	log(1 + 4.5 * RC + 4.5 * RM) − log(1 + 4.5 * NRA + 4.5 * NRI)

In [ ]:
set(activities[activities['Element ID'].isin(cmmv.Element)]['Element ID'])

In [ ]:
activities = activities.loc[(activities['Element ID'].isin(cmmv.Element)) & (activities['Scale ID'] == 'IM')]

In [ ]:
set(context[context['Element ID'].isin(cmmv.Element)]['Element ID'])

In [ ]:
context = context.loc[(context['Element ID'].isin(cmmv.Element)) & (context['Scale ID'] == 'CX')]

In [ ]:
# reverse 4.C.3.b.8	CX
context.loc[context['Element ID'] == '4.C.3.b.8', 'Data Value'] = 5 - context.loc[context['Element ID'] == '4.C.3.b.8', 'Data Value']

In [ ]:
set(abilities[abilities['Element ID'].isin(cmmv.Element)]['Element ID'])

In [ ]:
abilities = abilities[(abilities['Element ID'].isin(cmmv.Element)) & (abilities['Scale ID'] == 'IM')]

In [ ]:
df = pd.concat([abilities, context, activities])

In [ ]:
df['O*NET-SOC Code'] = df['O*NET-SOC Code'].str[:-3]

In [ ]:
for skill in set(cmmv.Skill):
    skill_items = cmmv[cmmv.Skill == skill].Element
    df.loc[df['Element ID'].isin(skill_items), 'Skill type'] = skill

In [ ]:
df_skills = df[['O*NET-SOC Code', 'Skill type','Data Value']].groupby(['O*NET-SOC Code', 'Skill type']).mean() / 5

In [ ]:
df_skills = df_skills.reset_index()

In [ ]:
xwalk = pd.read_excel('../data/skill_Bowen/crosswalk_bls.xlsx', sheet_name='xwalk')
xwalk_extra = pd.read_excel('../data/skill_Bowen/crosswalk_bls.xlsx', sheet_name='xwalk help')
xwalk = pd.concat([xwalk, xwalk_extra])

In [ ]:
df_skills = df_skills.merge(xwalk, on='O*NET-SOC Code', how='outer')

In [ ]:
df_skills = df_skills[['Skill type', 'Data Value', 'OCC_CODE']].groupby(['OCC_CODE', 'Skill type']).mean()

In [ ]:
df_skills

Routine index (RTI index)	(Autor and Dorn (2013))	log(1 + 4.5 * RC + 4.5 * RM) − log(1 + 4.5 * NRA + 4.5 * NRI)

In [ ]:
for occ in set(df_skills.index.get_level_values('OCC_CODE')):
    rm = df_skills.loc[(occ, 'Routine manual (RM)'), 'Data Value']
    nra = df_skills.loc[(occ, 'Non-routine analytical (NRA)'), 'Data Value']
    nri = df_skills.loc[(occ, 'Non-routine interactive (NRI)'), 'Data Value']
    rc = df_skills.loc[(occ, 'Routine cognitive (RC)'), 'Data Value']

    df_skills.loc[(occ, 'Routine index (RTI)'), 'Data Value'] = np.log10(1 + 4.5*rc + 4.5*rm) - \
                                                                        np.log10(1 + 4.5*nra + 4.5*nri)

In [ ]:
df_skills.to_csv('../data/skill_Bowen/skill_per_BLS_out.csv')

# apply to shock

In [ ]:
util_small_to_upp_caps = {'Electric bulk power transmission and control': 'Electric Bulk Power Transmission and Control',
 'Wind electric power generation': 'Wind Electric Power Generation',
 'Electric power distribution': 'Electric Power Distribution',
 'Solar electric power generation': 'Solar Electric Power Generation',
 '221200': 'Natural Gas Distribution',
 'Gas electric power generation': 'Gas Electric Power Generation',
 'Geothermal electric power generation':'Geothermal Electric Power Generation',
 'Nuclear electric power generation': 'Nuclear Electric Power Generation',
 '221300': 'Water, Sewage and Other Systems',
 'Other electric power generation': 'Other Electric Power Generation',
 'Hydroelectric power generation': 'Hydroelectric Power Generation',
 'Coal electric power generation': 'Coal Electric Power Generation',
 'Biomass electric power generation': 'Biomass Electric Power Generation'}
util_upp_to_small_caps = {v: k for k, v in util_small_to_upp_caps.items()}

In [ ]:
analysis = '../analysis/'
data = '../data/'
data_out = '../results/data_out/'
output_folder = '../results/data_out/results/'
fig_folder = '../results/figs/'
version = '3_oct_2022'
ylim = (-250000, 600000)

In [ ]:
df_skills = pd.read_csv(data + 'skill_Bowen/skill_per_BLS_out.csv', index_col=0)

In [ ]:
df_occ = pd.read_csv(data_out + 'occ_names_bls_minor_major.csv', index_col=0)
df_occ = df_occ[['OCC_CODE', 'OCC_TITLE', 'TOT_EMP']]

In [ ]:
ind_occ = pd.read_csv(data_out + 'bi-partite_emp_sum_elec.csv', index_col=0)
ind_occ.rename(index = util_upp_to_small_caps, inplace=True)

ind_occ_frac = ind_occ.T.div(ind_occ.sum(axis=1), axis=1).fillna(0.0)

In [ ]:
df_full = pd.DataFrame()
for yr in range(2021, 2048, 1):
    df = pd.read_csv(output_folder + 'industry_employment_effects_' + \
                                        str(yr) + '-'+ str(yr+1) + \
                                         '__' + version + '.csv', index_col=0)
    df['year'] = yr
    df_full = pd.concat([df_full, df])

df_full = df_full.reset_index().set_index(['index', 'year'])
df_try = df_full.unstack().T.copy()
df_try.index = df_try.index.droplevel(0)
df_try = df_try.groupby(df_try.index).sum()
df_try.loc[2020, :] = 0
df_try.sort_index(inplace=True)

cumul_plus = df_try[df_try > 0].sum()
cumul_min = -1 * df_try[df_try < 0].sum()

# cumul_plus = temp + perm
# cumul_min = lost + temp
dff = (cumul_plus - cumul_min)

perm = dff.clip(lower=0.0)
lost = (-1 * dff).clip(lower=0.0)
temp = (cumul_plus - perm)

ind_perm = ind_occ_frac @ perm
ind_lost = ind_occ_frac @ lost
ind_temp = ind_occ_frac @ temp

In [ ]:
skills_pivot = df_skills.reset_index().pivot_table(columns='OCC_CODE', index='Skill type', values='Data Value')

In [ ]:
df_profile = pd.concat([ind_perm, ind_temp, ind_lost, ind_occ.sum()], axis=1).rename(columns={0: 'perm', 1: 'temp', 2: 'lost', 3: 'tot'})

In [ ]:
skill_profile = skills_pivot @ df_profile

In [ ]:
skill_profile.div(skill_profile.sum(axis=0), axis=1).round(2)

In [ ]:
a = skills_pivot.apply(lambda x: df_profile.mul(x, axis=0).stack()).unstack()

In [ ]:
df_skill_occ = pd.concat({x: df_profile.mul(skills_pivot.loc[x], axis=0) for x in skills_pivot.index}, axis=1)

In [ ]:
df_skill_occ.stack()

In [ ]:
df_skill_and_weight = pd.concat([skills_pivot.T, df_profile], axis=1)

In [ ]:
df_skill_and_weight[['perm', 'temp', 'lost', 'tot']] = df_skill_and_weight[['perm', 'temp', 'lost', 'tot']].div(df_skill_and_weight[['perm', 'temp', 'lost', 'tot']].sum())

In [ ]:
ax = df_skill_and_weight[['Non-routine analytical (NRA)', 'Non-routine interactive (NRI)',
       'Non-routine manual (NRM)', 'Routine cognitive (RC)',
       'Routine index (RTI)', 'Routine manual (RM)']].plot(kind='hist', weights=df_skill_and_weight['tot'], subplots=True, alpha = 0.99, figsize=(12, 6), layout=(2,3), bins=25, histtype='step', legend=False, color='red', title=['Non-routine analytical (NRA)', 'Non-routine interactive (NRI)',
                                                                                                                                                                                                                                       'Non-routine manual (NRM)', 'Routine cognitive (RC)',
                                                                                                                                                                                                                                       'Routine index (RTI)', 'Routine manual (RM)'])

for tp, col in zip(['perm', 'lost', 'temp'], ['blue', 'green', 'black']):
    a = df_skill_and_weight[['Non-routine analytical (NRA)', 'Non-routine interactive (NRI)',
       'Non-routine manual (NRM)', 'Routine cognitive (RC)',
       'Routine index (RTI)', 'Routine manual (RM)']].plot(kind='hist', weights=df_skill_and_weight[tp], subplots=True, ax=ax, alpha = 0.99, bins=25, histtype='step', legend=False, color=col)
    
from matplotlib.lines import Line2D
custom_lines = [Line2D([0], [0], color='red', lw=2),
                Line2D([0], [0], color='blue', lw=2),
                Line2D([0], [0], color='green', lw=2),
                Line2D([0], [0], color='black', lw=2)]

ax[0][1].legend(custom_lines, ['All workers', 'Perm boost', 'Phase out', 'Temp boost'], loc='upper left')

# save figure
plt.savefig(fig_folder + 'skill_intensity.png', dpi=300, bbox_inches='tight')
# save figure pdf
plt.savefig(fig_folder + 'skill_intensity.pdf', dpi=300, bbox_inches='tight')
# save figure svg
plt.savefig(fig_folder + 'skill_intensity.svg', dpi=300, bbox_inches='tight')

plt.show()

In [ ]:
cols = ['Wind-capex', 'Solar-capex',
       'Natural gas-capex', 'Coal-capex', 'Biomass-capex', 'Geothermal-capex',
       'Hydro-capex', 'Battery storage-capex', 'T&D-capex', 'Biomass-opex', 'Coal-opex',
       'Solar-opex', 'Natural gas-opex', 'Geothermal-opex', 'Hydro-opex',
       'Nuclear-opex', 'Wind-opex', 'Transmission and distribution-opex',
       'Battery storage-opex']

df_full = pd.DataFrame()
for yr in range(2021, 2050, 1):
    df = pd.read_csv(output_folder + 'diff_baseline_employment_effects_' + \
                                        str(yr-1) + '-'+ str(yr) + \
                                         '__' + version + '.csv', index_col=0)
    df['year'] = yr
    df_full = pd.concat([df_full, df])

df_full = df_full.reset_index().set_index(['index', 'year'])
df_try = df_full.loc[:,cols].unstack().T.copy()
df_try.index = df_try.index.droplevel(0)
df_try = df_try.groupby(df_try.index).sum()
df_try.loc[2020, :] = 0
df_try.sort_index(inplace=True)

cumul_plus = df_try[df_try > 0].sum()
cumul_min = -1 * df_try[df_try < 0].sum()

# cumul_plus = temp + perm
# cumul_min = lost + temp
dff = (cumul_plus - cumul_min)

perm = dff.clip(lower=0.0)
lost = (-1 * dff).clip(lower=0.0)
temp = (cumul_plus - perm)

ind_perm = perm
ind_lost = lost
ind_temp = temp
split_year = 2034
end_year = 2038
# shock 2035 split
df_until2035 = df_full.loc[df_full.index.get_level_values('year').isin(range(2020, split_year + 1)), 'emp_tot'].groupby(df_full.loc[df_full.index.get_level_values('year').isin(range(2020, split_year + 1))].index.get_level_values('index')).sum()
df_after2035 = df_full.loc[df_full.index.get_level_values('year').isin(range(split_year + 1, end_year + 1)), 'emp_tot'].groupby(df_full.loc[df_full.index.get_level_values('year').isin(range(split_year + 1, end_year + 1))].index.get_level_values('index')).sum()

# rel employment 2035 split
merged_2035 = df_occ.merge(df_until2035, left_on='OCC_CODE', right_index=True, how='right')
merged_2035.set_index('OCC_CODE', inplace=True)
df_until2035_rel_emp = merged_2035.emp_tot / merged_2035.TOT_EMP

merged_2035 = df_occ.merge(df_after2035, left_on='OCC_CODE', right_index=True, how='right')
merged_2035.set_index('OCC_CODE', inplace=True)
df_after2035_rel_emp = merged_2035.emp_tot / merged_2035.TOT_EMP
ind_profs = pd.concat([ind_temp, ind_lost, ind_perm], axis=1).rename(columns={0: 'temp', 1: 'lost', 2: 'perm'})

ind_profs = ind_profs.merge(df_occ, right_on='OCC_CODE', left_index=True, how='outer')
# rel shock
ind_profs_rel_shock = ind_profs.copy()
ind_profs_rel_shock[['temp', 'lost', 'perm']] = ind_profs_rel_shock[['temp', 'lost', 'perm']].div(ind_profs_rel_shock[['temp', 'lost', 'perm']].sum())

# rel employment
ind_profs_rel_emp = ind_profs.copy()
ind_profs_rel_emp[['temp', 'lost', 'perm']] = ind_profs_rel_emp[['temp', 'lost', 'perm']].div(ind_profs_rel_emp.TOT_EMP, axis=0)

# RCA
ind_profs_rca = ind_profs.copy()
ind_profs_rca.TOT_EMP = ind_profs_rca.TOT_EMP / ind_profs_rca.TOT_EMP.sum()
ind_profs_rca[['temp', 'lost', 'perm']] = ind_profs_rca[['temp', 'lost', 'perm']].div(ind_profs_rca[['temp', 'lost', 'perm']].sum())# = ind_profs_rca.div(ind_profs_rca.sum())
# and
ind_profs_rca[['temp', 'lost', 'perm']] = ind_profs_rca[['temp', 'lost', 'perm']].div(ind_profs_rca.TOT_EMP, axis=0)
ind_profs = ind_profs.drop(['OCC_TITLE', 'TOT_EMP'], axis=1).set_index('OCC_CODE')
ind_profs_rel_emp = ind_profs_rel_emp.drop(['OCC_TITLE', 'TOT_EMP'], axis=1).set_index('OCC_CODE')
ind_profs_rel_shock = ind_profs_rel_shock.drop(['OCC_TITLE', 'TOT_EMP'], axis=1).set_index('OCC_CODE')
ind_profs_rca = ind_profs_rca.drop(['OCC_TITLE', 'TOT_EMP'], axis=1).set_index('OCC_CODE')

In [ ]:
display(ind_profs.head())
display(ind_profs_rel_emp.head())
display(ind_profs_rel_shock.head())
display(ind_profs_rca.head())

In [ ]:
thres = 0.01
display((ind_profs_rel_emp > thres).sum())
ind_profs_rel_emp_25 = ind_profs_rel_emp.copy()
ind_profs_rel_emp_25['idmax'] = ind_profs_rel_emp_25.idxmax(axis=1)
ind_profs_rel_emp_25 = ind_profs_rel_emp_25.loc[(ind_profs_rel_emp_25.drop('idmax', axis=1)>thres).any(axis=1)]
print(skills_pivot.drop(ind_profs_rel_emp_25.index, axis=1).shape[1])

In [ ]:
ind_profs_rel_emp.to_csv('../results/data_out/occs_archetypes_dynamic.csv')

In [ ]:
occ_arch = pd.read_csv('../data/omn/occ_archetypes_thresholds_relbase_2034_2038.csv')
occ_arch = occ_arch[['O*NET-SOC Code', 'Phase_out_r0.01', 'Permanent_boost_r0.01', 
                     'Temporary_boost_r0.01', 'Late_boost_r0.01']]
occ_arch.rename(columns = {'O*NET-SOC Code': 'OCC_CODE', 'Phase_out_r0.01': 'lost', 'Permanent_boost_r0.01': 'perm', 
                     'Temporary_boost_r0.01': 'temp', 'Late_boost_r0.01': 'late'}, inplace=True)
occ_arch.set_index('OCC_CODE', inplace=True)

In [ ]:
for idm in ['lost', 'perm', 'temp']:
    with pd.option_context("max_colwidth", 1000):
        #print(bls[bls.OCC_CODE.isin(ind_profs_rel_emp_25[ind_profs_rel_emp_25.idmax == idm].index)][['OCC_CODE', 'OCC_TITLE', 'A_MEAN']].to_latex())
        print(bls[bls.OCC_CODE.isin(occ_arch[occ_arch[idm] == 1].index)][['OCC_CODE', 'OCC_TITLE', 'A_MEAN']].to_latex())

In [ ]:
(ind_profs_rca > 5).sum()

ind_profs_rca_five = ind_profs_rca.copy()
ind_profs_rca_five['idmax'] = ind_profs_rca_five.idxmax(axis=1)

ind_profs_rca_five

# only those with rca > 5
ind_profs_rca_five = ind_profs_rca_five.loc[(ind_profs_rca_five.drop('idmax', axis=1)>5).any(axis=1)]

ind_profs_rca_five.idmax.value_counts()

skills_pivot.drop(ind_profs_rca_five.index, axis=1).shape[1]

In [ ]:
skills_pivot.index.to_list()

In [ ]:
#ax = skills_pivot.drop(ind_profs_rel_emp_25.index, axis=1) \
ax = skills_pivot.drop(occ_arch[occ_arch.sum(axis=1) > 0].index, axis=1) \
                .T.plot(kind='hist', color='red', alpha=1, subplots=True, 
                        legend=False, density=True, histtype='step', 
                        figsize=(10,5), layout=(2,3), ylim=(0,12), title=skills_pivot.index.to_list())
for item, col in zip(['perm', 'lost', 'temp'], ['blue', 'green', 'black']):
    #skills_pivot.loc[:, ind_profs_rel_emp_25[ind_profs_rel_emp_25.idmax == item].index
    skills_pivot.loc[:, occ_arch[occ_arch[item] == 1].index] \
                    .T.plot(kind='hist', ax=ax, subplots=True, 
                            color=col, alpha=1, legend=False, 
                            density=True, histtype='step')
    
from matplotlib.lines import Line2D
custom_lines = [Line2D([0], [0], color='red', lw=2),
                Line2D([0], [0], color='blue', lw=2),
                Line2D([0], [0], color='green', lw=2),
                Line2D([0], [0], color='black', lw=2)]

ax[0][0].legend(custom_lines, ['Other', 'Perm boost', 'Phase out', 'Temp boost'], loc='upper left')

# save figure
plt.savefig(fig_folder + 'skill_intensity_occ_archetypes.png', dpi=300, bbox_inches='tight')
# save figure pdf
plt.savefig(fig_folder + 'skill_intensity_occ_archetypes.pdf', dpi=300, bbox_inches='tight')
# save figure svg
plt.savefig(fig_folder + 'skill_intensity_occ_archetypes.svg', dpi=300, bbox_inches='tight')

In [ ]:
occ_arch

In [ ]:
df_names = pd.read_csv(data_out + 'occ_names_bls_minor_major.csv', index_col=0)

In [ ]:
# select from df_names all OCC_CODE from occ_arch with perm=1
otherr = 1
for archetype in ['lost', 'perm', 'temp', 'late']:
    print(archetype)
    perc_arch = df_names[df_names['OCC_CODE'].isin(occ_arch[occ_arch[archetype] == 1].index)].TOT_EMP.sum() / df_names.TOT_EMP.sum()
    print(perc_arch)
    otherr -= perc_arch
print(otherr)

In [ ]:
temp_occs = df_names[df_names['OCC_CODE'].isin(occ_arch[occ_arch['temp'] == 1].index)]
other_occs = df_names[df_names['OCC_CODE'].isin(occ_arch[occ_arch.sum(axis=1) == 0].index)]

In [ ]:
# temp jobs

for grouping in ['OCC_TITLE_major', 'OCC_TITLE_minor']:
    for gr_name in df_names[grouping].unique():
        gr_in_temp = temp_occs[temp_occs[grouping] == gr_name].TOT_EMP.sum()
        gr_in_total = df_names[df_names[grouping] == gr_name].TOT_EMP.sum()
        print(gr_name, '\n', gr_in_temp/gr_in_total)

In [ ]:
for grouping in ['OCC_TITLE_major', 'OCC_TITLE_minor']:
    for gr_name in df_names[grouping].unique():
        gr_in_others = other_occs[other_occs[grouping] == gr_name].TOT_EMP.sum()
        gr_in_total = df_names[df_names[grouping] == gr_name].TOT_EMP.sum()
        print(gr_name, '\n', gr_in_others/gr_in_total)

# Add network

In [ ]:
mixed = pd.read_csv('../data/omn/edgelist_cc_mobility_merge.csv')

In [ ]:
omn = mixed.pivot_table(index='OCC_source', columns='OCC_target', values='transition_prob_BLS_ASECnorm_corr', fill_value=0.0)
cc = mixed.pivot_table(index='OCC_source', columns='OCC_target', values='trans_prob_cc', fill_value=0.0)
mixed05 = mixed.pivot_table(index='OCC_source', columns='OCC_target', values='trans_merge_alpha05', fill_value=0.0)
mixed63 = mixed.pivot_table(index='OCC_source', columns='OCC_target', values='trans_merge_alphaweight', fill_value=0.0)

In [ ]:
# cc and omn do not have the same codes. 
print(set(cc.index) - set(omn.index))
print(set(omn.index) - set(cc.index))

## Assortativity

From Maria: Clustering and assortativity of attributes

Networkx only has inbuilt function for assortativity coefficient of integer values in unweighted networks. We code our assortativity coefficient for weighted and directed networks with scalar attributes. This coefficient is given by

$$
r = \frac{\sum_{ij} \left(A_{ij} - \frac{s_i^+ s_j^-}{W}\right)x_ix_j}{\sqrt{\sum_{ij} \left(k_i^-\delta_{ij} - \frac{s_i^+ s_j^+}{W}\right)x_ix_j\sum_{ij} \left(k_i^-\delta_{ij} - \frac{s_i^- s_j^-}{W}\right)x_ix_j} }
$$
where $A$ is the weighted adjacency matrix, $s_i^+$ and $s_i^-$ are the in and out strenghts of node $i$ given by
$$ s_i^+ = \sum_j A_{ij}$$
$$ s_i^- = \sum_i A_{ij}$$
and
$$W = \sum_{ij}A_{ij} $$
is the sum of the weights of all edges in the network.

In [ ]:
# function that gives assortativity coefficient (to double check I coded it for the
# definition and simplified formula we derive. They both give same results)

def delta_kron(i,j):
    if i == j:
        return 1
    else:
        return 0

def assortativity_weighted_directed(A, x, direction='both'):
    """
    Computes assortativity coefficient for weighted and directed networks
    A(np array): adjacency matrix of network
    x(np array): nodes' attributes
    NOTE error in standard deviatos, should be strenght instead of A
    """
    n = A.shape[0]
    W = A.sum()
    W_div = 1/W
    # outstregth
    s_plus = A.sum(axis=1)
    # instrength
    s_minus = A.sum(axis=0)
    # naming covariance and sf for + and -
    cov = 0
    sd_plus = 0
    sd_minus = 0
    for i in range(n):
        for j in range(n):
            if direction == 'both':
                cov += (A[i,j] - W_div * s_plus[i]*s_minus[j])*x[i]*x[j]
                sd_plus += (s_plus[i]*delta_kron(i,j) - W_div * s_plus[i]*s_plus[j])*x[i]*x[j]
                sd_minus += (s_minus[j]*delta_kron(i,j) - W_div * s_minus[i]*s_minus[j])*x[i]*x[j]
            if direction == 'out':
                cov += (A[i,j] - W_div * s_plus[i]*s_plus[j])*x[i]*x[j]
                sd_plus += (s_plus[i]*delta_kron(i,j) - W_div * s_plus[i]*s_plus[j])*x[i]*x[j]
                sd_minus += (s_plus[j]*delta_kron(i,j) - W_div * s_plus[i]*s_plus[j])*x[i]*x[j]
            if direction == 'in':
                cov += (A[i,j] - W_div * s_minus[i]*s_minus[j])*x[i]*x[j]
                sd_plus += (s_minus[i]*delta_kron(i,j) - W_div * s_minus[i]*s_minus[j])*x[i]*x[j]
                sd_minus += (s_minus[j]*delta_kron(i,j) - W_div * s_minus[i]*s_minus[j])*x[i]*x[j]
    return cov / np.sqrt(sd_plus * sd_minus)

def assortativity_weighted_directed_v2(A, x, direction = 'both'):
    """
    Computes assortativity coefficient for weighted and directed networks
    A(np array): adjacency matrix of network
    x(np array): nodes' attributes
    JB changes: following https://arxiv.org/pdf/2101.05389.pdf
    """
    
    n = A.shape[0]
    W = A.sum()
    W_div = 1/W
    # outstregth
    s_plus = A.sum(axis=1)
    # instrength
    s_minus = A.sum(axis=0)
    # average over in and out str
    mu_plus = (s_plus * x).sum() * W_div
    mu_minus = (s_minus * x).sum() * W_div
    # naming covariance and sf for + and -
    cov = 0
    sd_plus = 0
    sd_minus = 0
    for i in range(n):
        for j in range(n):
            if direction == 'both':
                cov += A[i,j] * (x[i] - mu_plus) * (x[j] - mu_minus)
                sd_plus += A[i,j] * (x[i] - mu_plus)** 2
                sd_minus += A[i,j] * (x[j] - mu_minus)** 2
            if direction == 'out':
                cov += A[i,j] * (x[i] - mu_plus) * (x[j] - mu_plus)
                sd_plus += A[i,j] * (x[i] - mu_plus)** 2
                sd_minus += A[i,j] * (x[i] - mu_plus)** 2
            if direction == 'in':
                cov += A[i,j] * (x[i] - mu_minus) * (x[j] - mu_minus)
                sd_plus += A[i,j] * (x[j] - mu_minus)** 2
                sd_minus += A[i,j] * (x[j] - mu_minus)** 2
    return cov / np.sqrt(sd_plus * sd_minus)

def assortativity_weighted_directed_v2_vectorized(A, x, direction='both'):
    """
    Computes assortativity coefficient for weighted and directed networks
    A(np array): adjacency matrix of network
    x(np array): nodes' attributes
    JB changes: following https://arxiv.org/pdf/2101.05389.pdf
    """

    n = A.shape[0]
    W = A.sum()
    W_div = 1/W
    # outstregth
    s_plus = A.sum(axis=1)
    # instrength
    s_minus = A.sum(axis=0)
    # average over in and out str
    mu_plus = (s_plus * x).sum() * W_div
    mu_minus = (s_minus * x).sum() * W_div

    cov = A@(x - mu_minus)@(x - mu_plus)
    sd_plus = np.sum(A.T@((x - mu_plus) ** 2))
    sd_minus = np.sum(A@((x - mu_minus) ** 2))
        
    return cov / np.sqrt(sd_plus * sd_minus)


def assortativity_weighted_directed_discrete_vectorized(A, x):
    """
    Computes assortativity coefficient for weighted and directed networks
    A(np array): adjacency matrix of network
    x(np array): nodes' discrete attributes, categorical
    JB changes: follows Eq 2 of Newman (2002): https://journals.aps.org/pre/pdf/10.1103/PhysRevE.67.026126,
    but it weighs everything
    """
    x = np.array(x)
    set_of_types = set(x)
    e = np.zeros((len(set_of_types), len(set_of_types)))
    for i,typeS in enumerate(set_of_types):
        for j,typeT in enumerate(set_of_types):
            A_ij = A[x == typeS].T[x == typeT]
            e[i,j] = A_ij.sum() / A.sum()
            
    a = e.sum(axis=0)
    b = e.sum(axis=1)
    
    r = (np.diag(e).sum() - (a*b).sum()) / (1 - (a*b).sum())
    return r

def assortativity_weighted_directed_discrete_vectorized_v2(A, x):
    """
    Computes assortativity coefficient for weighted and directed networks
    A(np array): adjacency matrix of network
    x(np array): nodes' discrete attributes, categorical
    JB changes: follows Eq 2 of Newman (2002): https://journals.aps.org/pre/pdf/10.1103/PhysRevE.67.026126,
    but it weighs everything
    """
    x = np.array(x)
    set_of_types = set(x)
    e = np.zeros((len(set_of_types), len(set_of_types)))
    for i,typeS in enumerate(set_of_types):
        for j,typeT in enumerate(set_of_types):
            A_ij = A[x == typeS].T[x == typeT]
            e[i,j] = A_ij.sum() / A.sum()
            
    e2_sum = np.linalg.matrix_power(e, 2).sum()
    
    r = (np.trace(e) - e2_sum) / (1 - e2_sum)

    return r

def shuffle_assort(A, x, categorical):
    np.random.shuffle(x)
    if categorical:
        return assortativity_weighted_directed_discrete_vectorized(A, x)
    else:
        return assortativity_weighted_directed_v2_vectorized(A, x)

from tqdm import tqdm
def assort_bounds(A, x, size = 100000, categorical = False):
    shuffles = [shuffle_assort(A, x, categorical) for i in tqdm(range(size))]
    shuffles.sort()
    bounds = [shuffles[int(size / 1000)], shuffles[int(size / 100)], shuffles[int(size / 20)],
              shuffles[-1 + int(size / -1000)], shuffles[-1 + int(size / -100)], shuffles[-1 + int(size / -20)]]
    vals = np.array(shuffles)
    return bounds, np.nanmean(vals), np.nanstd(vals)

Testing our algorithm with trivial cases and also real network and compare with networkx
We first test both our versions coincide and get right the trivial cases. We then test that our algorithm reproduces the assortative coefficinet for an unweighted undirected network where nodes have integer attributes.


In [ ]:
# Trivial cases
A_two_cliques = np.array([[0, 1, 0, 0], [1, 0, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0]])
x_assortative = [1,0.3, 0, 0]
x_dissasortative = [1,0, 1, 0]
x_mix = [1,0, 0, 0]
print("Assortative case, alg v1", assortativity_weighted_directed(A_two_cliques, x_assortative))
print("Assortative case, alg v2", assortativity_weighted_directed_v2(A_two_cliques, x_assortative))
print("Dissasortative case, alg v1", assortativity_weighted_directed(A_two_cliques, x_dissasortative))
print("Dissasortative case, alg v2", assortativity_weighted_directed_v2(A_two_cliques, x_dissasortative))
print("mix case, alg v1", assortativity_weighted_directed(A_two_cliques, x_mix))
print("mix case, alg v2", assortativity_weighted_directed_v2(A_two_cliques, x_mix))


In [ ]:
# Less trivial cases
A_two_connected_cliques = np.array([[0, 1, 1, 0], [1, 0, 0, 0], [0, 0, 0, 1], [1, 0, 1, 0]])
x_assortative = [1,1, 0, 0]
x_dissasortative = [1,0, 1, 0.2]
x_mix = [1,0, 0, 0]
print("Assortative case, alg v1    ", assortativity_weighted_directed(A_two_connected_cliques, x_assortative))
print("Assortative case, alg v2    ", assortativity_weighted_directed_v2(A_two_connected_cliques, x_assortative))
print("Assortative case, alg v2 vec", assortativity_weighted_directed_v2_vectorized(A_two_connected_cliques, x_assortative))
print("Dissasortative case, alg v1    ", assortativity_weighted_directed(A_two_connected_cliques, x_dissasortative))
print("Dissasortative case, alg v2    ", assortativity_weighted_directed_v2(A_two_connected_cliques, x_dissasortative))
print("Disassortative case, alg v2 vec", assortativity_weighted_directed_v2_vectorized(A_two_connected_cliques, x_dissasortative))
print("mix case, alg v1    ", assortativity_weighted_directed(A_two_connected_cliques, x_mix))
print("mix case, alg v2    ", assortativity_weighted_directed_v2(A_two_connected_cliques, x_mix))
print("mix case, alg v2 vec", assortativity_weighted_directed_v2_vectorized(A_two_connected_cliques, x_mix))

In [ ]:
# discrete trivial cases
A_two_cliques = np.array([[0, 1, 0, 0], [1, 0, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0]])
x_assortative = [1,1, 0, 0]
x_dissasortative = [1,0, 1, 0]
x_mix = [1,0, 0, 0]
print("Assortative case, alg v2", assortativity_weighted_directed_v2(A_two_cliques, x_assortative))
print("Dissasortative case, alg v2", assortativity_weighted_directed_v2(A_two_cliques, x_dissasortative))
print("mix case, alg v2", assortativity_weighted_directed_v2(A_two_cliques, x_mix))
print("Assortative case, alg disc", assortativity_weighted_directed_discrete_vectorized(A_two_cliques, x_assortative))
print("Dissasortative case, alg disc", assortativity_weighted_directed_discrete_vectorized(A_two_cliques, x_dissasortative))
print("mix case, alg disc", assortativity_weighted_directed_discrete_vectorized(A_two_cliques, x_mix))
print("Assortative case, alg disc v2", assortativity_weighted_directed_discrete_vectorized_v2(A_two_cliques, x_assortative))
print("Dissasortative case, alg disc v2", assortativity_weighted_directed_discrete_vectorized_v2(A_two_cliques, x_dissasortative))
print("mix case, alg disc v2", assortativity_weighted_directed_discrete_vectorized_v2(A_two_cliques, x_mix))

In [ ]:
# discrete less trivial cases
A_two_cliques = np.array([[0, 1, 1, 0], [1, 0, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0]])
x_assortative = [1,1, 2, 2]
x_dissasortative = [1,0, 1, 3]
x_mix = [1,2, 3, 3]
print("Assortative case, alg disc", assortativity_weighted_directed_discrete_vectorized(A_two_cliques, x_assortative))
print("Dissasortative case, alg disc", assortativity_weighted_directed_discrete_vectorized(A_two_cliques, x_dissasortative))
print("mix case, alg disc", assortativity_weighted_directed_discrete_vectorized(A_two_cliques, x_mix))
print("Assortative case, alg disc v2", assortativity_weighted_directed_discrete_vectorized_v2(A_two_cliques, x_assortative))
print("Dissasortative case, alg disc v2", assortativity_weighted_directed_discrete_vectorized_v2(A_two_cliques, x_dissasortative))
print("mix case, alg disc v2", assortativity_weighted_directed_discrete_vectorized_v2(A_two_cliques, x_mix))

In [ ]:
occ_arch['oth'] = (1 - occ_arch.sum(axis=1)).clip(lower=0)

In [ ]:
omn_no_diag = omn.copy()
v = omn_no_diag.values
np.fill_diagonal(v, 0)
omn_no_diag = pd.DataFrame(data=v, index=omn_no_diag.index, columns=omn_no_diag.columns)
omn_no_diag = omn_no_diag.div(omn_no_diag.sum(axis=1), axis=0).fillna(0.0)

cc_no_diag  = cc.copy()
v = cc_no_diag.values
np.fill_diagonal(v, 0)
cc_no_diag = pd.DataFrame(data=v, index=cc_no_diag.index, columns=cc_no_diag.columns)
cc_no_diag = cc_no_diag.div(cc_no_diag.sum(axis=1), axis=0).fillna(0.0)

mixed05_no_diag  = mixed05.copy()
v = mixed05_no_diag.values
np.fill_diagonal(v, 0)
mixed05_no_diag = pd.DataFrame(data=v, index=mixed05_no_diag.index, columns=mixed05_no_diag.columns)
mixed05_no_diag = mixed05_no_diag.div(mixed05_no_diag.sum(axis=1), axis=0).fillna(0.0)

mixed63_no_diag  = mixed63.copy()
v = mixed63_no_diag.values
np.fill_diagonal(v, 0)
mixed63_no_diag = pd.DataFrame(data=v, index=mixed63_no_diag.index, columns=mixed63_no_diag.columns)
mixed63_no_diag = mixed63_no_diag.div(mixed63_no_diag.sum(axis=1), axis=0).fillna(0.0)

In [ ]:
# assortativity by weighted and directed
#assortativity_weighted_directed_v2(A, x):
# assortativity with NetworkX assortativity calculation
#import networkx as nx

# for df, txt in list(zip([ind_profs_rca, ind_profs, ind_profs_rel_emp, ind_profs_rel_shock, ind_profs_binary_rca, ind_profs_binary_emp], 
#                     ['RCA', 'Abs', 'Rel to emp', 'Rel to shck', 'binary RCA > 5', 'binary emprel > 0.025'])):
assort_list = []
for df, txt in list(zip([ind_profs_rel_emp, occ_arch], 
                    ['Rel to emp', 'binary emprel_1perc'])):
    for direc in ['both']:#, 'out', 'in']:
        df = df[['temp', 'lost', 'perm']]
        print("")
        print(txt)
        print(direc)
        assort = pd.DataFrame(columns=['omn', 'cc', 'mix05', 'mix63'], index=['perm', 'temp', 'lost'])
        assort_shuffle = pd.DataFrame(columns=['omn', 'cc', 'mix05', 'mix63'], index=['perm', 'temp', 'lost'])
        for netw, name in list(zip([omn_no_diag, cc_no_diag, mixed05_no_diag, mixed63_no_diag], ['omn', 'cc', 'mix05', 'mix63'])):
            for att in ['perm', 'temp', 'lost']:
                assort.loc[att, name] = assortativity_weighted_directed_v2_vectorized(netw.to_numpy(), df.loc[netw.index, att].to_numpy(), direction=direc)
                x_resh = df.loc[netw.index, att].to_numpy()
                bounds, mean, std = assort_bounds(netw.to_numpy(), x_resh, size=100000)
                assort_shuffle.loc[att, name] = [b.round(3) for b in bounds] + [mean, std]

        assort = assort.astype(float)
        assort.rename(index={'lost': 'phaseout'}, inplace=True)
        assort.to_csv('../results/data_out/assortativity/assort_' + txt + '.csv')
        assort_list += [assort]
        print(assort.round(3).to_latex())
        
        #assort_shuffle = assort_shuffle.astype(float)
        assort_shuffle.rename(index={'lost': 'phaseout'}, inplace=True)
        assort_shuffle.to_csv('../results/data_out/assortativity/assort_bounds_mean_std_' + txt + '.csv')
        assort_list += [assort_shuffle]
        print(assort_shuffle.to_latex())

In [ ]:
for df, txt in list(zip([df_until2035_rel_emp, df_after2035_rel_emp], 
                    ['bef35_rel_emp', 'aft35_rel_emp'])):
    print("")
    print(txt)
    assort = pd.DataFrame(columns=['omn', 'cc', 'mix05', 'mix63'], index=[txt])
    assort_shuffle = pd.DataFrame(columns=['omn', 'cc', 'mix05', 'mix63'], index=[txt])
    for netw, name in list(zip([omn_no_diag, cc_no_diag, mixed05_no_diag, mixed63_no_diag], ['omn', 'cc', 'mix05', 'mix63'])):
        assort.loc[txt, name] = assortativity_weighted_directed_v2_vectorized(netw.to_numpy(), df.loc[netw.index].to_numpy())
        x_resh = df.loc[netw.index].to_numpy()
        bounds, mean, std  = assort_bounds(netw.to_numpy(), x_resh, size=100000)
        assort_shuffle.loc[txt, name] = [b.round(3) for b in bounds] + [mean, std]


    assort = assort.astype(float)
    assort.to_csv('../results/data_out/assortativity/assort_' + txt + '.csv')
    print(assort.round(3).to_latex())
    
    assort_shuffle.to_csv('../results/data_out/assortativity/assort_bounds_mean_std_' + txt + '.csv')
    print(assort_shuffle.to_latex())

In [ ]:
occ_discr = occ_arch.stack()
occ_discr = occ_discr[occ_discr == 1]

In [ ]:
occ_discr = occ_discr.reset_index().drop(0, axis=1).rename(columns={'level_1': 'type'}).set_index('OCC_CODE')

In [ ]:
occ_discr

In [ ]:
assort_list = []
for df, txt in list(zip([occ_discr], ['binary arch_dicr'])):
    df = df['type']
    print("")
    print(txt)
    assort = pd.DataFrame(columns=['omn', 'cc', 'mix05', 'mix63'], index=[txt])
    assort_shuffle = pd.DataFrame(columns=['omn', 'cc', 'mix05', 'mix63'], index=[txt])
    for netw, name in list(zip([omn_no_diag, cc_no_diag, mixed05_no_diag, mixed63_no_diag], ['omn', 'cc', 'mix05', 'mix63'])):
        assort.loc[txt, name] = assortativity_weighted_directed_discrete_vectorized(netw.to_numpy(), df.loc[netw.index].to_numpy())
        x_resh = df.loc[netw.index].to_numpy()
        bounds, mean, std = assort_bounds(netw.to_numpy(), x_resh, size=100000, categorical=True)
        assort_shuffle.loc[txt, name] = [b.round(3) for b in bounds] + [mean, std]

    assort = assort.astype(float)
    assort.rename(index={'lost': 'phaseout'}, inplace=True)
    assort.to_csv('../results/data_out/assortativity/assort_' + txt + '.csv')
    assort_list += [assort]
    print(assort.round(3).to_latex())

    #assort_shuffle = assort_shuffle.astype(float)
    assort_shuffle.rename(index={'lost': 'phaseout'}, inplace=True)
    assort_shuffle.to_csv('../results/data_out/assortativity/assort_bounds_mean_std_' + txt + '.csv')
    assort_list += [assort_shuffle]
    print(assort_shuffle.to_latex())